# Creating word vectors using word2vec: implementation in Spark MLlib

## Download data and create input output directories

### Here we show how to use Spark MLlib Word2Vec for generating word-features 
#### The data being used is the attack comments text data

This notebook will take take about 1-2 mins to finish on a Python 3 Spark kernel on a DSVM with Spark

MLlib Word2Vec: https://spark.apache.org/docs/2.2.0/mllib-feature-extraction.html#word2vec

In [47]:
!cd /home/remoteuser/notebooks/SparkML
!rm -r Outputs
!mkdir Outputs

!cd /home/remoteuser/notebooks/SparkML/Data
!rm -r text_data.zip text_data
!wget https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip
!unzip text_data.zip

!cd /home/remoteuser/notebooks/SparkML/pySpark

--2018-03-04 03:36:39--  https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip
Resolving activelearning.blob.core.windows.net (activelearning.blob.core.windows.net)... 13.77.184.64
Connecting to activelearning.blob.core.windows.net (activelearning.blob.core.windows.net)|13.77.184.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64260679 (61M) [application/x-zip-compressed]
Saving to: ‘text_data.zip’

100%[======================================>] 64,260,679  26.4MB/s   in 2.3s   

2018-03-04 03:36:42 (26.4 MB/s) - ‘text_data.zip’ saved [64260679/64260679]

Archive:  text_data.zip
   creating: text_data/
  inflating: text_data/aggression_data.csv  
  inflating: text_data/attack_data.csv  
  inflating: text_data/toxicity_data.csv  


## Set directory path for input data 
#### Input data is downloaded locally to a DSVM

In [48]:
# 1. Location of training data on 
text_file = "/home/remoteuser/notebooks/SparkML/Data/text_data/attack_data.csv"

## Set spark context and import necessary libraries

In [49]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, DataFrame, SparkSession
from pyspark.sql.functions import UserDefinedFunction, regexp_replace, trim, col, lower, lit, udf, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.ml.feature import Word2Vec, Word2VecModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.linalg import Vectors

import numpy as np
import datetime
import pandas as pd


sqlContext = SQLContext(sc)

## Data ingestion: Read in attack text data from .csv file

In [50]:
## READ IN DATA AND CREATE SPARK DATAFRAME FROM A CSV & AND MATERIALIZE IN MEMORY
text_df = spark.read.csv(path=text_file, header=True, inferSchema=True, sep=",")
text_df.cache();

## REGISTER DATA-FRAME AS A TEMP-TABLE IN SQL-CONTEXT
text_df.createOrReplaceTempView("text_table")

# COUNT NUMBER OF ROWS IN DATA FRAME
text_df.count()

115864

In [51]:
# SELECT ONE ROW FROM THE SPARK DATAFRAME
text_df.head(1)

[Row(rev_id=37675, comment="`-NEWLINE_TOKENThis is not ``creative``.  Those are the dictionary definitions of the terms ``insurance`` and ``ensurance`` as properly applied to ``destruction``.  If you don't understand that, fine, legitimate criticism, I'll write up ``three man cell`` and ``bounty hunter`` and then it will be easy to understand why ``ensured`` and ``insured`` are different - and why both differ from ``assured``.NEWLINE_TOKENNEWLINE_TOKENThe sentence you quote is absolutely neutral.  You just aren't familiar with the underlying theory of strike-back (e.g. submarines as employed in nuclear warfare) guiding the insurance, nor likely the three man cell structure that kept the IRA from being broken by the British.  If that's my fault, fine, I can fix that to explain.  But ther'es nothing ``personal`` or ``creative`` about it.NEWLINE_TOKENNEWLINE_TOKENI'm tired of arguing with you.  Re: the other article, ``multi-party`` turns up plenty, and there is more use of ``mutually`` t

In [52]:
### SELECT A FEW COLUMNS BASED ON WHICH WE FILTER
sqlStatement = """ SELECT distinct ns, split 
            FROM text_table 
            where split in ('train', 'test', 'dev') 
            order by ns, split """
spark.sql(sqlStatement).show()

+-------+-----+
|     ns|split|
+-------+-----+
|article|  dev|
|article| test|
|article|train|
|   user|  dev|
|   user| test|
|   user|train|
+-------+-----+



## Select and filter data set (only training articles)

In [53]:
### SELCT ONLY REV_ID AND COMMENT FIELDS, AND FILTER FOR TRAINING DATA AND ARTILES ONLY
sqlStatement = """ SELECT rev_id, comment 
            FROM text_table 
            where ns = 'article' and split = 'train' """
text_filtered_df = spark.sql(sqlStatement)

## CACHE NEW DATAFRAME IN MEMORY AND CREATE TEMPORARY TABLE
text_filtered_df.cache(); 
text_filtered_df.createOrReplaceTempView("text_filtered_table")

## COUNT NUMBER OF ROWS IN DATAFRAME AFTER FILTERING
text_filtered_df.count()

31253

## Lowercase COMMENT and remove some irrelevant words and punctuations

In [54]:
text_filtered_df2 = text_filtered_df.withColumn("comment1", lower(col("comment"))).\
    withColumn("comment2", regexp_replace("comment1", '-newline_token', "")).\
    withColumn("comment3", regexp_replace("comment2", 'newline_token', "")).\
    withColumn("comment_final", regexp_replace("comment3", '[^\w-_ ]', "")).\
    select('rev_id', 'comment_final')
    

# SELECT ONE ROW OF DATAFRAME AFTER 
text_filtered_df2.head(1)

[Row(rev_id=37675, comment_final='this is not creative  those are the dictionary definitions of the terms insurance and ensurance as properly applied to destruction  if you dont understand that fine legitimate criticism ill write up three man cell and bounty hunter and then it will be easy to understand why ensured and insured are different - and why both differ from assuredthe sentence you quote is absolutely neutral  you just arent familiar with the underlying theory of strike-back eg submarines as employed in nuclear warfare guiding the insurance nor likely the three man cell structure that kept the ira from being broken by the british  if thats my fault fine i can fix that to explain  but theres nothing personal or creative about itim tired of arguing with you  re the other article multi-party turns up plenty and there is more use of mutually than mutual  if i were to apply your standard id be moving mutual assured destruction to talk for not appealing to a reagan voters biases abo

## Tokenize COMMENT and remove stopwords

In [55]:
## DEFINE TOKENIZER AND STOPWORD REMOVER
tokenizer = Tokenizer(inputCol="comment_final", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtWords")

## TRANSFORM DATASET TO TOKENIZE AND REMOVE STOPWORDS
text_filtered_df3 = tokenizer.transform(text_filtered_df2)
text_filtered_df4 = remover.transform(text_filtered_df3)

## MATERIALIZE DATAFRAME IN MEMORY
text_filtered_df4.cache(); text_filtered_df4.count();
text_filtered_df4.show(5)

+------+--------------------+--------------------+--------------------+
|rev_id|       comment_final|               words|           filtWords|
+------+--------------------+--------------------+--------------------+
| 37675|this is not creat...|[this, is, not, c...|[creative, , dict...|
| 44816| the term standar...|[, the, term, sta...|[, term, standard...|
| 49851|true or false the...|[true, or, false,...|[true, false, sit...|
| 93890|this page will ne...|[this, page, will...|[page, need, disa...|
|103624|i removed the fol...|[i, removed, the,...|[removed, followi...|
+------+--------------------+--------------------+--------------------+
only showing top 5 rows



## DEFINE AND RUN WORD2VEC ON COMMENTS

MLlib Word2Vec parameters: https://spark.apache.org/docs/2.2.0/api/scala/index.html#org.apache.spark.mllib.feature.Word2Vec

In [56]:
model = None
window_size = 5
vector_size = 50
min_count = 5

## DEFINE WORD2VEC TRANSFORMER
word2Vec = Word2Vec(windowSize = window_size, vectorSize = vector_size, minCount = min_count, inputCol="filtWords", outputCol="result")

## FIT TRANSFORMER TO GENERATE FEATURES
model = word2Vec.fit(text_filtered_df4)

## Examine some words, and other words close to them from these feature neighborhood

In [57]:
model.findSynonyms("good", 20).select("word").head(4)

[Row(word='bad'),
 Row(word='wikipediaassume'),
 Row(word='luck'),
 Row(word='assume')]

## Examine how the vector features look like

In [58]:
word2vec_features = model.getVectors().select("*")
word2vec_features.head(1)

[Row(word='quotient', vector=DenseVector([-0.0053, 0.005, -0.0141, 0.0217, 0.0175, 0.0126, 0.0187, -0.0265, 0.004, -0.0223, 0.0091, 0.0126, 0.0131, -0.0088, -0.0258, -0.0024, 0.0051, 0.0102, 0.0235, -0.0028, -0.0242, 0.0047, -0.0054, 0.0166, 0.0009, 0.0268, 0.0179, -0.0047, 0.0031, 0.0125, 0.0127, -0.0038, 0.0117, -0.0043, -0.0238, 0.0106, -0.0289, -0.0064, -0.0123, -0.0053, -0.0018, 0.005, 0.0017, 0.0005, 0.0239, -0.0248, -0.0397, 0.022, 0.0088, 0.0056]))]

## Convert Spark DF to Pandas DF for output into a CSV file

In [59]:
word2vec_features_pdf = word2vec_features.toPandas()
word2vec_features_pdf.head(4)

,word,vector
0,quotient,"[-0.00533114839345, 0.00501936348155, -0.01409..."
1,incident,"[0.0544441044331, -0.00731786200777, 0.1287074..."
2,serious,"[-0.0265728700906, 0.0296862889081, -0.0313590..."
3,wgbh,"[0.0179173815995, 0.0335741974413, 0.007226068..."


## Save features in CSV file for subsequent steps

In [60]:
word2vec_features_pdf.to_csv('/home/remoteuser/notebooks/SparkML/Outputs/Word2Vec-Features.csv')

## Getting comment-level vectors from word-level vectors (averaging)

In [61]:
comment_vectors_df = model.transform(text_filtered_df4).select('rev_id','result')
comment_vectors_df.head(2)

[Row(rev_id=37675, result=DenseVector([0.0082, -0.0071, 0.0157, -0.0909, 0.027, -0.0496, 0.0008, -0.0288, 0.0032, 0.0279, -0.0248, 0.0324, 0.0443, 0.0007, 0.0277, 0.0036, 0.0136, -0.0455, -0.0064, 0.0321, 0.0276, -0.017, -0.0632, -0.004, -0.0205, -0.0301, 0.0031, 0.0796, -0.0117, 0.0063, 0.0142, 0.0277, -0.0569, 0.0596, -0.0351, 0.0615, 0.0386, -0.0099, 0.0155, -0.0365, -0.0126, -0.0222, 0.0584, 0.0329, -0.0064, -0.0027, 0.0637, 0.0007, -0.0114, 0.0059])),
 Row(rev_id=44816, result=DenseVector([0.0274, -0.0178, 0.0082, -0.0921, 0.0176, -0.0476, 0.0055, -0.0293, -0.0218, 0.0229, -0.0133, 0.0264, 0.0513, 0.0085, 0.0349, -0.0014, 0.0155, -0.0698, -0.0015, 0.0316, 0.0418, -0.0274, -0.0768, 0.0224, 0.0006, -0.024, -0.0154, 0.0918, -0.0431, 0.008, -0.0073, 0.0305, -0.0344, 0.0837, -0.0224, 0.0552, 0.0374, -0.0126, 0.0273, -0.0351, 0.0128, -0.039, 0.0374, 0.0291, 0.0234, -0.0086, 0.0545, -0.0094, -0.0097, -0.0128]))]